In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('test').getOrCreate()

In [5]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import format_number

def crosstabPercentage(df, col1, col2):
    ctabDf = df.crosstab(col1, col2)
    ctabCol = col1 + '_' + col2
    ctabNewCol = col1 + col2.title()
    ctabDf = ctabDf.withColumn(ctabNewCol, ctabDf[ctabCol])\
                                         .orderBy(ctabNewCol).drop(ctabCol)
    # Strip extra whitespaces from column name
    for column in ctabDf.columns:
        columnStripped = column.strip()
        if column != columnStripped:
            ctabDf = ctabDf.withColumn(column.strip(), ctabDf[column])\
                                         .drop(column)

#     ctabDf = ctabDf.withColumn('percentage-of->50K', ctabDf['>50K']/(ctabDf['<=50K']+ctabDf['>50K'])*100)
#     return ctabDf
    ctabDf.show()

In [64]:
import math
from functools import reduce

df = spark.createDataFrame([
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("a", 1), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("c", -50),
    ("c", -50),
    ("c", -50),
    ("c", -50),
    ("c", -50),
    ("c", -50),
    ("c", -50),
    ("c", -50),
    ("c", -50),
    ("c", -50),
    ("c", -50),
    ("c", -50),
], ["group", "power"])

# High mutual information indicates a large reduction in uncertainty; 
# low mutual information indicates a small reduction; 
# and zero mutual information between two random variables means the 
# variables are independent.
def calcMutualInformation(df, *columns):
    from scipy.stats import entropy
    n = df.count()
    individualProbs = {}
    # Key: column, value: list of distinct values
    columnDistinctVals = {column: [] for column in columns}
    for column in columns:
        aggr = df.groupby(column).count()
        rows = aggr.withColumn('prob', (col('count') / n)).collect()
        for row in rows:
            val = row[column]
            individualProbs[(column, val, )] = row['prob']
            columnDistinctVals[column].append(val)
    
    aggr = df.groupby(*columns).count()
    rows = aggr.withColumn('prob', (col('count') / n)).collect()
    jointProbs = {}
    for row in rows:
        vals = tuple([(column, row[column]) for column in columns])
        prob = row['prob']
        jointProbs[vals] = prob
    
    mutualInformation = 0
    for k, v in jointProbs.items():
        jointProb = v
        indProbs = [individualProbs[ind] for ind in k]
        g = jointProb * math.log(jointProb / reduce((lambda x, y: x * y), indProbs))
        mutualInformation += g

    return mutualInformation

calcMutualInformation(df, 'group', 'power')

1.073383620336505

In [25]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType

from pyspark.sql.types import *
import pandas as pd

df = spark.createDataFrame([
    ("a", 0), ("a", 1), ("b", 30), ("b", -50)
], ["group", "power"])

def below_threshold(group="group", power="power"):
    @pandas_udf("struct<group: string, sum: int>", PandasUDFType.GROUPED_MAP)
    def below_threshold_(df):
        df = pd.DataFrame(
           df.groupby(group).apply(df[power].sum()))
        df.reset_index(inplace=True, drop=False)
        return df

    return below_threshold_

df.groupBy("group").apply(below_threshold()).show()

Py4JJavaError: An error occurred while calling o868.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 45 in stage 95.0 failed 1 times, most recent failure: Lost task 45.0 in stage 95.0 (TID 2802, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 260, in dump_stream
    for series in iterator:
  File "<string>", line 1, in <lambda>
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 97, in wrapped
    result = f(pd.concat(series, axis=1))
  File "<ipython-input-25-8ed7f0752e6f>", line 15, in below_threshold_
  File "/opt/conda/lib/python3.6/site-p

In [20]:
from pyspark.sql.functions import col, lit
from pyspark.ml.linalg import Vectors

# from pyspark.sql.window import Window
# from pyspark.sql.functions import count, rank, min

# windowSpec =  Window.orderBy('id2', 'id3')
# windowSpec = windowSpec.rowsBetween(Window.unboundedPreceding, Window.currentRow)
# # df.select(count('id3').over(w).alias('count')).show()
# df.select(min(labelCol).over(w)).show()

# df.crosstab("id1", "id2").show()
# crosstabPercentage(df, "id1", "id2")

# df2 = df.groupby(labelCol, "id2").count()
# count = df.count()
# df2.withColumn('percentage', col('count') / count).show()

In [17]:
grouped = df.groupby("id")
grouped

In [ ]:
grouped.

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest
from pyspark.sql.functions import min, max

data = [(0.0, Vectors.dense(0.5, 10.0, 10.0)),
        (0.0, Vectors.dense(1.5, 20.0, 10.0)),
        (1.0, Vectors.dense(1.5, 30.0, 20.0)),
        (0.0, Vectors.dense(3.5, 30.0, 10.0)),
        (0.0, Vectors.dense(3.5, 40.0, 10.0)),
        (1.0, Vectors.dense(3.5, 40.0, 20.0))]
df = spark.createDataFrame(data, ["label", "features"])

r = ChiSquareTest.test(df, "features", "label").head()
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))


pValues: [0.687289278791,0.682270330336,0.0143058784354]
degreesOfFreedom: [2, 3, 1]
statistics: [0.75,1.5,6.0]


In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = [(2, Vectors.dense(0.5, 10.0)),
        (0, Vectors.dense(1.5, 20.0)),
        (1, Vectors.dense(1.5, 30.0)),
        (2, Vectors.dense(3.5, 30.0)),
        (1, Vectors.dense(3.5, 40.0)),
        (1, Vectors.dense(3.5, 40.0))]
data = spark.createDataFrame(data, ["label", "features"])

# Split the data into train and test
splits = data.randomSplit([0.6, 0.4], 1234)
# train = splits[0]
# test = splits[1]

train = data

# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [2, 5, 4, 3]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(train)

# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))